In [1]:
import requests    # download the book 
import re          #`re` (regular expressions) library to remove unwanted characters and information from the text
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import contractions
from itertools import combinations
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import networkx as nx
from time import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohsenrahimi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import requests    # download the book 
book_id = 1342
url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    with open("book.txt", "w", encoding="utf-8") as f:
        f.write(response.text)
    print("Book downloaded successfully.")
else:
    print("Failed to download the book.")

Book downloaded successfully.


In [3]:
with open("book.txt", "r", encoding="utf-8") as f:
    book_content = f.read()                        #`book_content` now contains the entire text of the book

In [4]:
# Check the header and footer of the book
print("The header is :\n",book_content[:500])

print("\nThe footer is :\n",book_content[-500:])

The header is :
 ï»¿The Project Gutenberg eBook of Pride and prejudice, by Jane Austen

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
usin

The footer is :
 by copyright in
the U.S. unless a copyright notice is included. Thus, we do not
necessarily keep eBooks in compliance with any particular paper
edition.

Most people start at our website which has the main PG search
facility: www.gutenberg.org

This website includes information about Project Gutenberg-tm,
including how to make donations to the Project Gutenberg Literary
Archive Foundation, how to help produce our new eBooks, and how to
subscribe to our email n

In [5]:
#Remove page numbers, headers, and footers:

book_content = re.sub(r"^(.*?)\*{3} START OF (.*?)\*{3}", "", book_content, flags=re.DOTALL)  # Remove header
book_content = re.sub(r"\*{3} END OF (.*?)\*{3}(\s|.)*$", "", book_content, flags=re.DOTALL)  # Remove footer

In [6]:
# Check the content of the book after removing header and footer

print("The header is :\n",book_content[:300])
print("\nThe footer is :\n",book_content[-300:])

The header is :
 





                            [Illustration:

                             GEORGE ALLEN
                               PUBLISHER

                        156 CHARING CROSS ROAD
                                LONDON

                             RUSKIN HOUSE
                                   ]


The footer is :
 e means of uniting them.

                            [Illustration:

                                  THE
                                  END
                                   ]




             CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
                  TOOKS COURT, CHANCERY LANE, LONDON.




It seems that the regular expressions used to remove the header and footer didn't work as expected. Let's try an alternative approach using the actual content of the book.


In [7]:
# Identify the start phrase
with open("book.txt", "r", encoding="utf-8") as f:
    book_content = f.read()                        #`book_content` now contains the entire text of the book
 
# In below in each time I printed 10,000 characters to find out the starting point

#print(book_content[1:30000])
print(book_content[30000:40000])
#print(book_content[40000:50000])

                                              78

Heading to Chapter XIV.                                               84

âProtested that he never read novelsâ                                 87

Heading to Chapter XV.                                                89

Heading to Chapter XVI.                                               95

âThe officers of the ----shireâ                                       97

âDelighted to see their dear friend againâ                           108

Heading to Chapter XVIII.                                            113

âSuch very superior dancing is not often seenâ                       118

âTo assure you in the most animated languageâ                        132

Heading to Chapter XX.                                               139

âThey entered the breakfast-roomâ                                    143

Heading to Chapter XXI.                                              146

âWalked back with themâ           

In [8]:
print("The starting phrase is `Chapter I.]`")

The starting phrase is `Chapter I.]`


In [9]:
# Identify the end phrase of the actual content
#print(book_content[-5000:])
#print(book_content[-10000:-5000])
#print(book_content[-15000:-10000])
print(book_content[-20000:-15000])

 had never before fallen in
her way. By Elizabethâs instructions she began to comprehend that a
woman may take liberties with her husband, which a brother will not
always allow in a sister more than ten years younger than himself.

Lady Catherine was extremely indignant on the marriage of her nephew;
and as she gave way to all the genuine frankness of her character, in
her reply to the letter which announced its arrangement, she sent him
language so very abusive, especially of Elizabeth, that for some time
all intercourse was at an end. But at length, by Elizabethâs persuasion,
he was prevailed on to overlook the offence, and seek a reconciliation;
and, after a little further resistance on the part of his aunt, her
resentment gave way, either to her affection for him, or her curiosity
to see how his wife conducted herself; and she condescended to wait on
them at Pemberley, in spite of that pollution which its woods had
received, not merely from the presence of such a mistress, but 

In [10]:
print("The ending phrase is `THE END`")

The ending phrase is `THE END`


#### Identify the start and end phrases of the actual content:

For this specific book (Pride and Prejudice), the actual content starts after the phrase `CHAPTER II` and 
ends before the phrase `THE END`. I can use these phrases to extract the main content: 

In [11]:
with open("book.txt", "r", encoding="utf-8") as f:
    book_content = f.read()                        #`book_content` now contains the entire text of the book
 

start_phrase = "Chapter I.]"
end_phrase = "END OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE "

start_index = book_content.find(start_phrase)
end_index = book_content.find(end_phrase)

book_content = book_content[start_index:end_index]

print("Check the content of the book after removing header and footer\n")
print("The content is stated from:\n\n",book_content[:500])
print("\nThe content is ended at:\n\n",book_content[-500:])

Check the content of the book after removing header and footer

The content is stated from:

 Chapter I.]


It is a truth universally acknowledged, that a single man in possession
of a good fortune must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding families, that he is considered as the rightful
property of some one or other of their daughters.

âMy dear Mr. Bennet,â said his lady to him one day, âhave you heard that
Netherfield Park is let at 

The content is ended at:

 most intimate terms. Darcy,
as well as Elizabeth, really loved them; and they were both ever
sensible of the warmest gratitude towards the persons who, by bringing
her into Derbyshire, had been the means of uniting them.

                            [Illustration:

                                  THE
                                  END
                                   

In [12]:


# Add an extra "CHAPTER" keyword at the end of the book for the last chapter
#adding the "CHAPTER" keyword at the end. It seems like you have added it to ensure that the last chapter is 
#included correctly when splitting.

book_content_with_last_chapter = book_content + "CHAPTER"

# Split the book into chapters using the "CHAPTER" keyword
chapters = book_content_with_last_chapter.split("CHAPTER")

# Remove the first element, which is just the text before the first chapter
#chapters.pop(0)

# Remove the last element, which is just the text after the last chapter
chapters.pop()

# Print the number of chapters and a sample chapter
print(f"Total chapters: {len(chapters)}")


Total chapters: 60


In [13]:
# print the start and end of first and last chapter respectively

print("\nChapter 1 starts from:\n", chapters[0][:300])
print("\nChapter 59 ends at:\n\n", chapters[59][-500:])


Chapter 1 starts from:
 Chapter I.]


It is a truth universally acknowledged, that a single man in possession
of a good fortune must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding famil

Chapter 59 ends at:

 most intimate terms. Darcy,
as well as Elizabeth, really loved them; and they were both ever
sensible of the warmest gratitude towards the persons who, by bringing
her into Derbyshire, had been the means of uniting them.

                            [Illustration:

                                  THE
                                  END
                                   ]




             CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
                  TOOKS COURT, CHANCERY LANE, LONDON.

*** 


In [14]:
import pandas as pd

# Calculate the number of words in each chapter
word_counts = [len(chapter.split()) for chapter in chapters]

# Create a DataFrame with the chapter number and word count
chapter_word_counts = pd.DataFrame({"Chapter": range(1, len(chapters) + 1), "Word Count": word_counts})
#print(chapter_word_counts)

In [15]:
import re
#Remove special characters and numbers,    we use the `re.sub()` function. 

#This example removes all non-alphabetic characters except spaces and newline characters:

#book_content_clean = re.sub(r"[^a-zA-Z\s\n]", "", book_content)


This modified regular expression retains alphabetic characters, spaces, newline characters, 
and the following punctuation marks: ".", ",", ";", "!", "?", "'", and "\"". This way, we'll preserve 
sentence boundaries, abbreviations, and other useful punctuation marks while still removing other special 
characters and numbers that might not be relevant to your analysis.

In [16]:
import re
# Function to clean the content of a chapter
def clean_chapter_content(chapter_content):
    return re.sub(r"[^a-zA-Z\s\n.,;!?'\"]", "", chapter_content)

# Apply the cleaning function to all chapters
chapters_clean = [clean_chapter_content(chapter) for chapter in chapters]

# Print a sample cleaned chapter
print("Cleaned Chapter 1 as below:\n", chapters_clean[0][:300])

Cleaned Chapter 1 as below:
 Chapter I.


It is a truth universally acknowledged, that a single man in possession
of a good fortune must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding famili


In [17]:
# apply the `contractions.fix()` function to each chapter (e.g., converting "don't" to "do not")

import contractions

chapters_no_contractions = [contractions.fix(chapter) for chapter in chapters_clean]

print("Chapter 1 without contractions:\n", chapters_no_contractions[0][:500])

Chapter 1 without contractions:
 Chapter I.


It is a truth universally acknowledged, that a single man in possession
of a good fortune must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding families, that he is considered as the rightful
property of some one or other of their daughters.

My dear Mr. Bennet, said his lady to him one day, have you heard that
Netherfield Park is let at last?

Mr.


In [18]:
# Concatenate the lines in each chapter
chapters_continuous = [' '.join(chapter.splitlines()) for chapter in chapters_no_contractions]

# Print a sample chapter
print("Chapter 1:\n", chapters_continuous[0][:500])

Chapter 1:
 Chapter I.   It is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife.  However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered as the rightful property of some one or other of their daughters.  My dear Mr. Bennet, said his lady to him one day, have you heard that Netherfield Park is let at last?  Mr.


For each chapter, it splits the text into lines using `splitlines()`, then joins the lines back together using the `join()` function with a space as the separator. The result is a list of chapters, named `chapters_continuous`, where each chapter is a single continuous string without line breaks.

In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mohsenrahimi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
from nltk.tokenize import sent_tokenize
# Apply sentence tokenization to all chapters
chapters_sentences = [sent_tokenize(chapter) for chapter in chapters_continuous]

# Printthe first 10 tokenized sentences from Chapter I
chapters_sentences[0][:10]

['Chapter I.',
 'It is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife.',
 'However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered as the rightful property of some one or other of their daughters.',
 'My dear Mr. Bennet, said his lady to him one day, have you heard that Netherfield Park is let at last?',
 'Mr. Bennet replied that he had not.',
 'But it is, returned she; for Mrs. Long has just been here, and she told me all about it.',
 'Mr. Bennet made no answer.',
 'Do not you want to know who has taken it?',
 'cried his wife, impatiently.',
 'You want to tell me, and I have no objection to hearing it.']

This code uses a list comprehension to apply the `sent_tokenize()` function to each chapter in the `chapters_continuous` list, creating a new list of chapters with tokenized sentences named `chapters_sentences`.

To retain the dialogue structure in the text, you can use regular expressions to identify and extract the dialogues before applying the preprocessing steps. Here's a step-by-step guide on how to do this:

1. Identify dialogue patterns in the text: Dialogues are usually enclosed in double quotes (") or single quotes (') in books. You can create a regular expression pattern to match these dialogues.

python
import re

#### Regular expression pattern for dialogues enclosed in double quotes
dialogue_pattern = r'"(.*?)"'


2. Extract dialogues from the chapters: You can use the `re.findall()` function to extract all the dialogues from each chapter.

python
##### Extract dialogues from all chapters
chapters_dialogues = [re.findall(dialogue_pattern, chapter) for chapter in chapters_continuous]


3. Replace dialogues in the chapters with placeholders: To keep the dialogue structure intact, replace the dialogues in each chapter with a placeholder (e.g., `DIALOGUE_PLACEHOLDER`). This will help you re-insert the dialogues after preprocessing.

python
##### Replace dialogues with placeholders
chapters_no_dialogues = [re.sub(dialogue_pattern, "DIALOGUE_PLACEHOLDER", chapter) for chapter in chapters_continuous]


4. Apply preprocessing steps to `chapters_no_dialogues`: Perform the preprocessing steps on the `chapters_no_dialogues` list, which now has placeholders instead of actual dialogues.

5. Re-insert dialogues: After preprocessing, replace the `DIALOGUE_PLACEHOLDER` placeholders with the original dialogues stored in the `chapters_dialogues` list.

python
def replace_dialogue_placeholders(text, dialogues):
    for dialogue in dialogues:
        text = text.replace("DIALOGUE_PLACEHOLDER", dialogue, 1)
    return text

chapters_preprocessed = [replace_dialogue_placeholders(preprocessed_chapter, dialogues) for preprocessed_chapter, dialogues in zip(chapters_preprocessed_without_dialogues, chapters_dialogues)]


Now, you have a list of preprocessed chapters (`chapters_preprocessed`) with the dialogue structure retained. This will help you analyze character interactions and relationships more effectively.

No I do not do this stage


Alright, if you do not wish to retain the dialogue structure, you can continue with your preprocessing steps without considering the dialogues. The previous steps I provided for sentence tokenization and other preprocessing tasks do not specifically address dialogue structure, so you can proceed with those steps as they are.



### NER - Name Entity Recognition 
I have a list called `chapters_sentences`, where each element represents a list of tokenized sentences for each chapter, now I can process them using the `BERT-based NER model from Hugging Face's Transformers library`.
I will iterate through each chapter and process each sentence with the NER pipeline to find the named entities in each sentence.

In [21]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Load a pre-trained BERT-based NER model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Create a NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
# Process each chapter's sentences
for chapter_idx, sentences in enumerate(chapters_sentences[:1]):
    print(f"Processing Chapter {chapter_idx + 1}")
    
    for sentence in sentences:
        # Extract named entities from the sentence
        entities = ner_pipeline(sentence)
        
        # Print the named entities
        for entity in entities:
            print(entity["word"], entity["entity"])
    print()
    

Processing Chapter 1
Ben I-PER
##net I-PER
Net I-LOC
##her I-LOC
##field I-LOC
Park I-LOC
Ben I-PER
##net I-PER
Long I-PER
Ben I-PER
##net I-PER
George I-PER
Allen I-PER
Long I-PER
Net I-LOC
##her I-LOC
##field I-LOC
England I-LOC
Morris I-PER
Michael I-MISC
Bing I-ORG
##ley I-ORG
Ben I-PER
##net I-PER
Bing I-PER
##ley I-PER
Bing I-PER
##ley I-PER
William I-PER
Lucas I-PER
Bing I-PER
##ley I-PER
Liz I-PER
##zy I-PER
Liz I-PER
##zy I-PER
Jane I-PER
Lydia I-PER
Liz I-PER
##zy I-PER
Ben I-PER
##net I-PER
Ben I-PER
##net I-PER
Ben I-PER
##net I-PER
George I-PER
Allen I-PER
Bing I-PER
##ley I-PER
George I-PER
Allen I-PER



It seems like the NER model has successfully identified some of the character names in Chapter II. 
However, some names are not correctly identified, and there are some extra characters ("##") in the output. 
These extra characters are a result of BERT's tokenization process, which splits words into subwords when needed.
So I will post-process the NER output to merge these subwords and provide a cleaner output of the character names.


Now I need to filter out some entities such as, Mrs, Miss and King since I'm looking just for character's name

In [ ]:
def merge_subwords(entities):
    merged_entities = []
    for entity in entities:
        if entity["word"].startswith("##"):
            if len(merged_entities) > 0:
                merged_entities[-1]["word"] += entity["word"][2:]
        else:
            merged_entities.append(entity)
    return merged_entities



In [ ]:
common_titles = ["Mrs", "Mr", "Miss", "Dr", "Sir","King"]

def is_valid_entity(entity):
    return entity["entity"] == "I-PER" and entity["word"] not in common_titles

for chapter_idx, sentences in enumerate(chapters_sentences[:3]):
    print(f"Processing Chapter {chapter_idx + 1}")
    
    for sentence in sentences:
        # Extract named entities from the sentence
        entities = ner_pipeline(sentence)
        
        # Merge subwords and clean up the named entities
        merged_entities = merge_subwords(entities)
        
        # Filter out unwanted entity types and common titles
        filtered_entities = [entity for entity in merged_entities if is_valid_entity(entity)]
        
        # Print the named entities
        for entity in filtered_entities:
            print(entity["word"], entity["entity"])
    print()    

## co-occurrence matrix

In [23]:
%history


import requests    # download the book 
import re          #`re` (regular expressions) library to remove unwanted characters and information from the text
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import contractions
from itertools import combinations
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import networkx as nx
from time import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
nltk.download('stopwords')
import requests    # download the book 
book_id = 1342
url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    with open("book.txt", "w", encoding="utf-8") as f:
        f.write(response.text)
    print("Book downloaded successfully.")
else:
    print("Failed to download the book.")
with open("book.txt", "r", encoding

In [24]:
 pip install jupytext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
jupyter nbextension install --py jupytext --user
   jupyter nbextension enable --py jupytext --user
   jupyter serverextension enable --py jupytext --user

SyntaxError: invalid syntax (1291984116.py, line 1)

In [26]:
jupyter nbextension install --py jupytext --user

SyntaxError: invalid syntax (304255055.py, line 1)